In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Activation, Add, Multiply
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

In [7]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/full_df.csv')

In [8]:
target_column = '현재수요(MW)'

data.set_index('기준일시',inplace=True)

features = data.drop(columns=[target_column])
target = data[target_column]

In [9]:
scaler = MinMaxScaler()
features = scaler.fit_transform(features)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)

In [10]:
#input shape: temporal dimensioni of data
#Dilated Causal Convolution Layer: capture temporal dependencies at different time scales
def wavenet(input_shape, num_filters, num_layers, dilation_rates):
    inputs = Input(shape=input_shape)
    
    skips = []
    x = inputs
    for dilation_rate in dilation_rates:
      #causal: model does not have access to future information during training and inference
        x = Conv1D(filters=num_filters, kernel_size=2, dilation_rate=dilation_rate, padding='causal')(x)
        tanh_out = Activation('tanh')(x)
        sigmoid_out = Activation('sigmoid')(x)
        x = Multiply()([tanh_out, sigmoid_out])
        x = Conv1D(filters=num_filters, kernel_size=1, padding='causal')(x)
        skips.append(x)
        x = Add()(skips)
        # capture both local and global dependencies in the data
    
    x = Activation('relu')(x)
    x = Conv1D(filters=num_filters, kernel_size=1)(x)
    x = Activation('relu')(x)
    x = Conv1D(filters=num_filters, kernel_size=1)(x)
    outputs = Conv1D(filters=1, kernel_size=1)(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [11]:
input_shape = (X_train.shape[1], 1)
num_filters = 9 #변경가능
num_layers = 3 #변경가능
dilation_rates = [2**i for i in range(num_layers)]

model = wavenet(input_shape, num_filters, num_layers, dilation_rates)
model.summary()

model.compile(optimizer=Adam(), loss='mean_absolute_percentage_error')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 52, 1)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 52, 9)        27          ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 52, 9)        0           ['conv1d[0][0]']                 
                                                                                                  
 activation_1 (Activation)      (None, 52, 9)        0           ['conv1d[0][0]']                 
                                                                                              

In [13]:
# Training loop
from tqdm import tqdm
num_epochs = 1 #변경가능
batch_size = 32 #변경가능
num_batches = len(X_train) // batch_size
for epoch in tqdm(range(num_epochs)):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    # Shuffle data before each epoch
    indices = np.random.permutation(len(X_train))
    shuffled_X_train = X_train[indices]
    shuffled_y_train = y_train[indices]
    
    # Mini-batch training
    for batch in range(num_batches):
        start = batch * batch_size
        end = start + batch_size
        
        batch_X = np.expand_dims(shuffled_X_train[start:end], axis=-1)
        batch_y = shuffled_y_train[start:end]
        
        # Train model on current batch
        model.train_on_batch(batch_X, batch_y)
        
    # Evaluate model on test set
    test_X = np.expand_dims(X_test, axis=-1)
    predictions = model.predict(test_X)
    
    # Flatten predictions and y_test arrays
    predictions = predictions.flatten()
    y_test = y_test.to_numpy().flatten()

    predictions = predictions[:len(y_test)]
    mape = mean_absolute_percentage_error(y_test, predictions)

    print("Mean Absolute Percentage Error (MAPE):", mape)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/1
6470/6470 [==============================] - 20s 3ms/step


100%|██████████| 1/1 [07:14<00:00, 434.57s/it]

Mean Absolute Percentage Error (MAPE): 0.1202409602408508


In [14]:
mape

0.1202409602408508